# Prerequisites

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.40.0


# Initialize Workspace

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

test
test2
australiaeast
6470fd9a-c001-4c06-9c58-79e82b8d01aa


# Dataset

In [3]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

datastore = ws.get_default_datastore()
# Dataset.File.upload_directory(src_dir='BBCNewsSummary_test', 
#                               target=DataPath(datastore, "datasets/BBCNewsSummary_test"),
#                               overwrite=True, show_progress=True
#                              ) 
dataset = Dataset.File.from_files(path=(datastore, 'datasets/BBCNewsSummary/'))

# Environment

In [4]:
%%writefile conda_dependencies_summary.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip=21.3.1
- pip:
  - azureml-defaults
  - torch==1.6.0
  - pandas
  - transformers==4.18.0
  - numpy
  - sklearn
  - datasets
  - nltk
  - rouge-score

Overwriting conda_dependencies_summary.yml


In [5]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = './conda_dependencies_summary.yml')

# Specify a GPU base image
pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [6]:
from azureml.core import  Experiment
experiment = Experiment(workspace=ws, name='test-datastore')

Text summarization

In [7]:
from azureml.core import ScriptRunConfig
sum_run = ScriptRunConfig(
        source_directory='./',
        script='text_sum3.py',
        compute_target='GPU56gtrainmodel',
        arguments=[
            '--data_path', dataset.as_named_input('input').as_mount(),
            # '--output_dir', output,
            # '--output_dir','./outputs'
            ],
        environment=pytorch_env
    ) 

sum_run = experiment.submit(sum_run)
sum_run.wait_for_completion(show_output=False)
summary_model = sum_run.register_model(model_name = 'summary_fine_tune', model_path = 'outputs/')
print(summary_model.name, summary_model.id, summary_model.version, sep = '\t')


Text classification

In [ ]:
from azureml.core import ScriptRunConfig

Class_run = ScriptRunConfig(
        source_directory='./',
        script='text_class_v3.py',
        compute_target='GPU56gtrainmodel',
        arguments=[
            '--data_path', dataset.as_named_input('input').as_mount()            
            ],
        environment=pytorch_env
    ) 

Class_run = experiment.submit(Class_run)
Class_run.wait_for_completion(show_output=False)
Class_model = Class_run.register_model(model_name = 'Class_fine_tune', model_path = 'outputs/')
print(Class_model.name, Class_model.id, Class_model.version, sep = '\t')

# Deploy

In [ ]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("torch==1.6.0")
environment.python.conda_dependencies.add_pip_package("transformers==4.18.0")
environment.python.conda_dependencies.add_pip_package("pandas")
environment.python.conda_dependencies.add_pip_package("nltk")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

Deploy LocalWebservice

In [ ]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
Class_model= Model(ws, 'Class_fine_tune')

inference_config = InferenceConfig(entry_script="score4.py",
                                     environment=environment)                                
                                  
# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [Class_model,summary_model], inference_config, deployment_config)

local_service.wait_for_deployment()

In [ ]:
import json
result = local_service.run(input_data=json.dumps({'data': 'Spam e-mails tempt net shoppers','length_summary':100,'amount_recom':3}))
# result = local_service.run(input_data=json.dumps({'data': 'Spam e-mails tempt net shoppers'}))

print(result)

In [ ]:
sample_input = json.dumps({'length_summary':100,'amount_recom':3,
    'data': '''Spam e-mails tempt net shoppers

Computer users across the world continue to ignore security warnings about spam e-mails and are being lured into buying goods, a report suggests.

More than a quarter have bought software through spam e-mails and 24% have bought clothes or jewellery. As well as profiting from selling goods or services and driving advertising traffic, organised crime rings can use spam to glean personal information. The Business Software Alliance (BSA) warned that people should "stay alert". "Many online consumers don't consider the true motives of spammers," said Mike Newton, a spokesperson for the BSA which commissioned the survey.

"By selling software that appears to be legitimate in genuine looking packaging or through sophisticated websites, spammers are hiding spyware without consumers' knowledge. "Once the software is installed on PCs and networks, information that is given over the internet can be obtained and abused."

The results also showed that the proportion of people reading - or admitting to reading - and taking advantage of adult entertainment spam e-mails is low, at one in 10. The research, which covered 6,000 people in six countries and their attitudes towards junk e-mails, revealed that Brazilians were the most likely to read spam. A third of them read unsolicited junk e-mail and 66% buy goods or services after receiving spam.

The French were the second most likely to buy something (48%), with 44% of Britons taking advantage of products and services. This was despite 38% of people in all countries being worried about their net security because of the amount of spam they get. More than a third of respondents said they were concerned that spam e-mails contained viruses or programs that attempted to collect personal information. "Both industry and the media have helped to raise awareness of the issues that surround illegitimate e-mail, helping to reduce the potential financial damage and nuisance from phishing attacks and spoof websites," said William Plante, director of corporate security and fraud protection at security firm Symantec. "At the same time, consumers need to continue exercising caution and protect themselves from harm with a mixture of spam filters, spyware detection software and sound judgement."'''
})

local_service.run(sample_input)

In [ ]:
local_service.delete()

In [ ]:
# local_service.update(models=[model1],
#                      inference_config=inference_config,
#                      deployment_config=deployment_config)


Deploy AciWebservice

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

# inference_config = InferenceConfig(entry_script="pytorch_score.py", environment=pytorch_env)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2)
                                            #    tags={'data': 'birds',  'method':'transfer learning', 'framework':'pytorch'},
                                            #    description='Classify turkey/chickens using transfer learning with PyTorch')

service = Model.deploy(workspace=ws, 
                           name='aci-birds1', 
                           models=[Class_model,summary_model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

In [ ]:
text_test='''Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.'''

# Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.

# Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.

# TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.'''


In [ ]:
import json
sample_input = json.dumps({'data': text_test,'length_summary':100,'amount_recom':3})
service.run(sample_input)
# service.run(sample_input)

In [ ]:
service.delete()